In [6]:
import pyarrow.parquet as pq
import json
import numpy as np
import pandas as pd
import pyarrow as pa
from joblib import Parallel, delayed
from tqdm_joblib import tqdm_joblib

/opt/homebrew/lib/python3.9/site-packages/tqdm_joblib/__init__.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [7]:
df = pd.read_csv('asl-signs/train.csv')
df



,path,participant_id,sequence_id,sign
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie
...,...,...,...,...
94472,train_landmark_files/53618/999786174.parquet,53618,999786174,white
94473,train_landmark_files/26734/999799849.parquet,26734,999799849,have
94474,train_landmark_files/25571/999833418.parquet,25571,999833418,flower
94475,train_landmark_files/29302/999895257.parquet,29302,999895257,room


In [8]:
label_map = json.load(open('asl-signs/sign_to_prediction_index_map.json'))
print(label_map)

{'TV': 0, 'after': 1, 'airplane': 2, 'all': 3, 'alligator': 4, 'animal': 5, 'another': 6, 'any': 7, 'apple': 8, 'arm': 9, 'aunt': 10, 'awake': 11, 'backyard': 12, 'bad': 13, 'balloon': 14, 'bath': 15, 'because': 16, 'bed': 17, 'bedroom': 18, 'bee': 19, 'before': 20, 'beside': 21, 'better': 22, 'bird': 23, 'black': 24, 'blow': 25, 'blue': 26, 'boat': 27, 'book': 28, 'boy': 29, 'brother': 30, 'brown': 31, 'bug': 32, 'bye': 33, 'callonphone': 34, 'can': 35, 'car': 36, 'carrot': 37, 'cat': 38, 'cereal': 39, 'chair': 40, 'cheek': 41, 'child': 42, 'chin': 43, 'chocolate': 44, 'clean': 45, 'close': 46, 'closet': 47, 'cloud': 48, 'clown': 49, 'cow': 50, 'cowboy': 51, 'cry': 52, 'cut': 53, 'cute': 54, 'dad': 55, 'dance': 56, 'dirty': 57, 'dog': 58, 'doll': 59, 'donkey': 60, 'down': 61, 'drawer': 62, 'drink': 63, 'drop': 64, 'dry': 65, 'dryer': 66, 'duck': 67, 'ear': 68, 'elephant': 69, 'empty': 70, 'every': 71, 'eye': 72, 'face': 73, 'fall': 74, 'farm': 75, 'fast': 76, 'feet': 77, 'find': 78, '

In [85]:
def frame_check(df):
    min = df['frame'].min()
    max = df['frame'].max()
    if (max-min) < 5:
        return False
    else: 
        return True

In [86]:
def frame_count(df):
    min = df['frame'].min()
    max = df['frame'].max()
    return max, min

In [9]:
import timeit

In [11]:

def extract_file(filename):
    # start = timeit.default_timer()
    df_tmp = pq.read_table(f'asl-signs/{filename}').to_pandas()
    if not frame_check(df_tmp):
        return np.array([0]), False
    df_tmp = df_tmp.fillna(0)
    face_landmarks = [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 
                     291, 146, 91, 181, 84, 17, 314, 405, 321, 375,  
                     191, 80, 81, 82, 13, 312, 311, 310, 415, 308, 
                     95, 88, 178, 87, 14, 317, 402, 318, 324]
    pose_landmarks = [11, 12, 13, 14, 15, 16,
                      17, 18, 19, 20, 21, 22]
    x_list = []
    y_list = []
    z_list = []
    all_list = []
    hand = ['left_hand', 'right_hand']
    tmp_frame = 0
    max_f, min = frame_count(df_tmp)
    frames = max_f-min+1
    df_tmp = df_tmp.query('type == @hand or (type == "face" & landmark_index == @face_landmarks) or (type == "pose" & landmark_index == @pose_landmarks) ')
    if frames > 20:
        buff = max_f-20
        df_tmp = df_tmp.query('frame >= @buff')
        for i in range(max_f-19, max_f+1):
            tmp_df = df_tmp.query('frame == @i')
            all_list.append(np.array([tmp_df['x'].astype(np.float32).to_numpy(), tmp_df['y'].astype(np.float32).to_numpy(), tmp_df['z'].astype(np.float32).to_numpy()]).flatten())
    else:
        for i in range(min, max_f+1):
            tmp_df = df_tmp.query('frame == @i')
            all_list.append(np.array([tmp_df['x'].astype(np.float32).to_numpy(), tmp_df['y'].astype(np.float32).to_numpy(), tmp_df['z'].astype(np.float32).to_numpy()]).flatten())
    ret = np.array(all_list).shape[0]
    if ret < 20:
        add_list = [[0 for i in range(279)] for j in range(20-ret)]
        all_list = add_list + all_list
    # print(timeit.default_timer() - start)
    shp = np.array(all_list).shape
    try:
        if shp[0] != 20 or shp[1]!= 279:
            print(np.array(all_list).shape)
    except:
        return np.array([0]), False       
    else:
        if shp[0] != 20 or shp[1]!= 279:
            print(np.array(all_list).shape)
        return np.asarray(all_list), True


In [10]:
def convert_label(raw):
    return label_map[raw]

In [92]:
# Parallelism

path = df['path'].to_numpy().astype(str)
raw_label = df['sign'].to_numpy().astype(str)
with tqdm_joblib(desc="Label conversion", total=94477) as progress_bar:
    label = Parallel(n_jobs=-1)(delayed(convert_label)(i) for i in raw_label)
with tqdm_joblib(desc="Data conversion", total=94477) as progress_bar:
    data, cond = zip(*Parallel(n_jobs=-1)(delayed(extract_file)(i) for i in path))


Label conversion:   0%|          | 0/94477 [00:00<?, ?it/s]

Data conversion:   0%|          | 0/94477 [00:00<?, ?it/s]

/var/folders/vt/q06w72ls4kz7lg_wsqqw2f0h0000gp/T/ipykernel_85603/3475926203.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/var/folders/vt/q06w72ls4kz7lg_wsqqw2f0h0000gp/T/ipykernel_85603/3475926203.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/var/folders/vt/q06w72ls4kz7lg_wsqqw2f0h0000gp/T/ipykernel_85603/3475926203.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must spe

In [109]:
# Removing false data
new_df = pd.DataFrame(data=[data, cond, label]).T
new_df.columns = ['data', 'cond', 'label']
new_df

,data,cond,label
0,"[[0.5064058, 0.50869834, 0.50817716, 0.5090293...",True,25
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True,232
2,"[[0.40560728, 0.40628085, 0.40590265, 0.404934...",True,48
3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True,23
4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True,164
...,...,...,...
94472,"[[0.49037775, 0.49069953, 0.49301714, 0.493240...",True,238
94473,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True,108
94474,"[[0.51680005, 0.515573, 0.51561564, 0.5125802,...",True,86
94475,"[[0.47087848, 0.47033817, 0.4701829, 0.4695914...",True,188


In [110]:
new_df = new_df[new_df.cond != False]
new_df = new_df.drop(['cond'], axis=1)
new_df = new_df.reset_index(drop=True)
new_df

,data,label
0,"[[0.5064058, 0.50869834, 0.50817716, 0.5090293...",25
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",232
2,"[[0.40560728, 0.40628085, 0.40590265, 0.404934...",48
3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",23
4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",164
...,...,...
94099,"[[0.49037775, 0.49069953, 0.49301714, 0.493240...",238
94100,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",108
94101,"[[0.51680005, 0.515573, 0.51561564, 0.5125802,...",86
94102,"[[0.47087848, 0.47033817, 0.4701829, 0.4695914...",188


In [12]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


In [120]:
X_data = [data for data in new_df['data'].to_numpy()]
X_data = np.array(X_data)
print(X_data.shape)
Y_data = [label for label in new_df['label'].to_numpy()]
Y_data = np.array(Y_data)
Y_data = to_categorical(Y_data).astype(int)
print(Y_data.shape)

(94104, 20, 279)
(94104, 250)


In [128]:
np.save("feature_data.npy", X_data)
np.save("label_data.npy", Y_data)

In [13]:
X_data = np.load("feature_data.npy")
Y_data = np.load("label_data.npy")

In [14]:
X_train, X, y_train, y = train_test_split(X_data, Y_data, test_size=0.1, shuffle=True)
X_test, X_val, y_test, y_val = train_test_split(X, y, test_size=0.5, shuffle=True)

In [15]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)

(84693, 20, 279)
(84693, 250)
(4705, 20, 279)
(4705, 250)
(4706, 20, 279)
(4706, 250)


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten, GRU, Conv1D
from tensorflow.keras.callbacks import TensorBoard, Callback
from tensorflow.keras.optimizers import Adam

In [17]:
class trainingCallback(Callback):
  def on_epoch_end(self, epoch, logs={}):
    
    # Check accuracy
    # if(logs.get('categorical_accuracy') < 0.95  and logs.get('loss') < 0.35 and logs.get('val_loss') < 0.35):
    if((logs.get('categorical_accuracy') > 0.95) or (logs.get('categorical_accuracy') > 0.92  and logs.get('loss') > logs.get('val_loss'))):
      # Stop if threshold is met
      print("\nAccuracy grater than 0.92 so cancelling training!")
      self.model.stop_training = True

# Instantiate class
callbacks = trainingCallback()

In [23]:
# LSTM Model
inputs = tf.keras.layers.Input(shape=(20,279))
x_1, w, h = tf.keras.layers.LSTM(256, return_sequences=False, activation='relu', return_state=True)(inputs)
x = tf.keras.layers.Dropout(0.2)(x_1)
# x = tf.keras.layers.LSTM(128, return_sequences=False, activation='relu')(x, initial_state=[w, h])
concat = tf.keras.layers.concatenate([x, w, h])
# flatten = tf.keras.layers.Flatten(concat)
x = tf.keras.layers.Dense(512, activation='relu')(concat)
x = tf.keras.layers.Dropout(0.2)(x)
# x = tf.keras.layers.Dense(64, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.5)(x)
out = tf.keras.layers.Dense(250, activation='softmax', name='outputs')(x)
model_LSTM = tf.keras.Model(inputs, out)
model_LSTM.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 20, 279)]    0           []                               
                                                                                                  
 lstm_2 (LSTM)                  [(None, 256),        548864      ['input_3[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                                  
 dropout_4 (Dropout)            (None, 256)          0           ['lstm_2[0][0]']                 
                                                                                            

In [24]:
model_LSTM.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [25]:
model_train = model_LSTM.fit(X_train, y_train, epochs=200, batch_size=64,validation_data=(X_val,y_val), callbacks=[callbacks])


Epoch 1/200
1324/1324 [==============================] - 54s 40ms/step - loss: 4.9970 - categorical_accuracy: 0.0291 - val_loss: 4.3978 - val_categorical_accuracy: 0.0829
Epoch 2/200
1324/1324 [==============================] - 58s 44ms/step - loss: 4.1398 - categorical_accuracy: 0.1076 - val_loss: 3.8806 - val_categorical_accuracy: 0.1528
Epoch 3/200
1324/1324 [==============================] - 55s 42ms/step - loss: 3.7750 - categorical_accuracy: 0.1592 - val_loss: 3.5856 - val_categorical_accuracy: 0.2012
Epoch 4/200
1324/1324 [==============================] - 58s 44ms/step - loss: 3.4880 - categorical_accuracy: 0.2058 - val_loss: 3.3766 - val_categorical_accuracy: 0.2378
Epoch 5/200
1324/1324 [==============================] - 57s 43ms/step - loss: 3.2698 - categorical_accuracy: 0.2440 - val_loss: 3.1624 - val_categorical_accuracy: 0.2835
Epoch 6/200
1324/1324 [==============================] - 53s 40ms/step - loss: 3.1221 - categorical_accuracy: 0.2716 - val_loss: 3.0807 - val_cat

KeyboardInterrupt: 

In [37]:
# LSTM Model
inputs = tf.keras.layers.Input(shape=(20,279), name='inputs')
x_1= tf.keras.layers.LSTM(256, return_sequences=True, activation='relu', name='lstm_1')(inputs)
x = tf.keras.layers.Dropout(0.2, name='drop_1')(x_1)
x, w, h= tf.keras.layers.LSTM(512, return_sequences=False, activation='relu', return_state=True, name='lstm_2')(x)
# x = tf.keras.layers.LSTM(128, return_sequences=False, activation='relu')(x, initial_state=[w, h])
concat = tf.keras.layers.concatenate([x, w, h], name='concat_1')
# flatten = tf.keras.layers.Flatten(concat)
x = tf.keras.layers.Dense(1024, activation='relu', name='dense_1')(concat)
x = tf.keras.layers.Dropout(0.2, name='drop_2')(x)
# x = tf.keras.layers.Dense(64, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.5)(x)
out = tf.keras.layers.Dense(250, activation='softmax', name='outputs')(x)
model_LSTM_2 = tf.keras.Model(inputs, out)
model_LSTM_2.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputs (InputLayer)            [(None, 20, 279)]    0           []                               
                                                                                                  
 lstm_1 (LSTM)                  (None, 20, 256)      548864      ['inputs[0][0]']                 
                                                                                                  
 drop_1 (Dropout)               (None, 20, 256)      0           ['lstm_1[0][0]']                 
                                                                                                  
 lstm_2 (LSTM)                  [(None, 512),        1574912     ['drop_1[0][0]']                 
                                 (None, 512),                                               

In [38]:
model_LSTM_2.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_train = model_LSTM_2.fit(X_train, y_train, epochs=10, batch_size=64,validation_data=(X_val,y_val), callbacks=[callbacks])


Epoch 1/10
1324/1324 [==============================] - 201s 151ms/step - loss: 5.1402 - categorical_accuracy: 0.0175 - val_loss: 4.7274 - val_categorical_accuracy: 0.0429
Epoch 2/10
 598/1324 [============>.................] - ETA: 1:56 - loss: 4.6446 - categorical_accuracy: 0.0514